In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from torch.utils.data import Dataset
import pickle
import pandas as pd

In [2]:
if(torch.cuda.is_available()):
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")
print(dev)

cpu


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import csv

database_folder = "/content/drive/MyDrive/DataBase"
f = os.listdir(database_folder)
training_data = []

for s in f:
  with open('/content/drive/MyDrive/DataBase/'+s, newline='') as csvfile:
    print(s)
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        training_data.append([float(item) for item in row])
    print(len(training_data))

database_Florence (Longitude _ 11.2558 _ Latitude _ 43.7696)_europe21.csv
17841
database_Paris (Longitude _ 2.3522 _ Latitude _ 48.8566)_europe21.csv
100429
database_Paris (Longitude _ 2.3522 _ Latitude _ 48.8566)_france21.csv
172733
database_Athènes (Longitude _ 23.7275 _ Latitude _ 37.9838)_europe21.csv
417196
database_Prague (Longitude _ 14.4378 _ Latitude _ 50.0755)_europe21.csv
643632
database_Berlin (Longitude  13.405  Latitude  52.52)_europe.csv
748103
database_france_from_france21.csv
2814188
database_france_from_france21_2.csv
4829088


In [5]:
print(len(training_data ))

4829088


In [6]:

with open("/content/drive/MyDrive/Data_Winds/objet_wind_data_2021.pickle", "rb") as f:
    wind_data = pickle.load(f)
    data = wind_data['data']
import datetime



In [7]:
print(wind_data['metadata'])

{'dataset': 'NOAA', 'grid': {'time': [datetime.datetime(2021, 1, 1, 0, 0), datetime.datetime(2021, 1, 1, 6, 0), datetime.datetime(2021, 1, 1, 12, 0), datetime.datetime(2021, 1, 1, 18, 0), datetime.datetime(2021, 1, 2, 0, 0), datetime.datetime(2021, 1, 2, 6, 0), datetime.datetime(2021, 1, 2, 12, 0), datetime.datetime(2021, 1, 2, 18, 0), datetime.datetime(2021, 1, 3, 0, 0), datetime.datetime(2021, 1, 3, 6, 0), datetime.datetime(2021, 1, 3, 12, 0), datetime.datetime(2021, 1, 3, 18, 0), datetime.datetime(2021, 1, 4, 0, 0), datetime.datetime(2021, 1, 4, 6, 0), datetime.datetime(2021, 1, 4, 12, 0), datetime.datetime(2021, 1, 4, 18, 0), datetime.datetime(2021, 1, 5, 0, 0), datetime.datetime(2021, 1, 5, 6, 0), datetime.datetime(2021, 1, 5, 12, 0), datetime.datetime(2021, 1, 5, 18, 0), datetime.datetime(2021, 1, 6, 0, 0), datetime.datetime(2021, 1, 6, 6, 0), datetime.datetime(2021, 1, 6, 12, 0), datetime.datetime(2021, 1, 6, 18, 0), datetime.datetime(2021, 1, 7, 0, 0), datetime.datetime(2021, 1

In [8]:
data_std = 1
dir_number = 8

def grid_time(t):
    t = t
    t = t/3600
    t = t + 1 + 2
    t = t - t%6
    t = t - 1
    return (t+1)/6

def grid_lat(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5
    l = l/2
    l = l +90
    l = l*2

    return (l/5)%73

def grid_long(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5

    return (l/5)%144

def int_array(arr):
    return [int(i) for i in arr]

def grid_array(arr):
    return int_array([grid_long(arr[0]),grid_lat(arr[1]),grid_long(arr[2]),grid_lat(arr[3]),grid_time(arr[4]),int(arr[5])])

def get_winds_at_t(data,time,dtime):
    tensor = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    return tensor

def input_label(training_data,dtime,i,data):
    array = grid_array(training_data[i])
    time = array[4]
    long_start = array[0]
    lat_start = array[1]
    long_end = array[2]
    lat_end = array[3]
    alt = array[5]
    winds = data[time:time+dtime,:,:,:]
    winds = np.transpose(winds,(0,4,1,2,3))
    winds = np.arctan2(winds[:,0:1,:,:,:],winds[:,1:2,:,:,:])/(3.1415/2)
    pos = np.zeros((dtime,1,winds.shape[2],winds.shape[3],winds.shape[4]))
    for i in range(pos.shape[2]):
        pos[0,0,i,long_end,lat_end] = -1
        pos[0,0,i,long_start,lat_start] = 1
    input = np.concatenate((winds,pos),axis=1)
    action = np.arctan2(data[time,alt,long_start,lat_start,1],data[time,alt,long_start,lat_start,0])
    action = int(np.round(((action+np.pi)*360/(2*np.pi))/(360/dir_number))%dir_number)

    return (input,action)

In [9]:
class CustomDataset(Dataset):
    def __init__(self, entries, dtime, data, transform=None, target_transform=None):
        self.entries = entries
        self.transform = transform
        self.target_transform = target_transform
        self.dtime = dtime
        self.data = data


    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        inp,lab = input_label(self.entries,self.dtime,idx,self.data)
        inp = np.pad(inp,((0,0),(0,0),(0,0),(2,2),(0,0)),'wrap')
        return inp, lab

In [10]:
trainset = CustomDataset(training_data,1,data)
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)


In [11]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
print(a.shape)


torch.Size([32, 1, 2, 17, 148, 73])


In [12]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dropout):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        out = self.dropout(out)
        return out


class ResBlockBN(nn.Module):
    def __init__(self, channels, kernel_size, stride, padding, dropout):
        super(ResBlockBN, self).__init__()
        reduced_channel = int(channels/4)
        self.conv1 = nn.Conv2d(channels, reduced_channel, 1, 1, 0)
        self.bn1 = nn.BatchNorm2d(reduced_channel)
        self.conv2 = nn.Conv2d( reduced_channel,  reduced_channel, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm2d(reduced_channel)
        self.conv3 = nn.Conv2d( reduced_channel,  channels, 1, 1, 0)
        self.bn3 = nn.BatchNorm2d(channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()
        self.bn3.weight.data.fill_(1)
        self.bn3.bias.data.zero_()

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out += identity
        out = self.relu(out)
        out = self.dropout(out)
        return out

class ProjectedResBlockBN(nn.Module):
    def __init__(self, channels, kernel_size, stride, padding, dropout):
        super(ProjectedResBlockBN, self).__init__()
        reduced_channels = int(channels/2)
        self.conv1 = nn.Conv2d(channels, reduced_channels, 1, 1, 0)
        self.bn1 = nn.BatchNorm2d(reduced_channels)
        self.conv2 = nn.Conv2d( reduced_channels,  reduced_channels, kernel_size, 2 , padding)
        self.bn2 = nn.BatchNorm2d(reduced_channels)
        self.conv3 = nn.Conv2d( reduced_channels,  channels*2, 1, 1, 0)
        self.bn3 = nn.BatchNorm2d(channels*2)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

        self.projector = nn.Conv2d( channels,  channels*2, kernel_size, 2 , padding)
        self.bnProj = nn.BatchNorm2d(channels*2)


        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()
        self.bn3.weight.data.fill_(1)
        self.bn3.bias.data.zero_()

    def forward(self, x):
        identity = x
        identity = self.projector(identity)
        identity = self.bnProj(identity)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)
        out = self.dropout(out)
        return out



class ProjectedResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dropout):
        super(ProjectedResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, 2, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.projector = nn.Conv2d(in_channels,out_channels,kernel_size, stride = 2 , padding = padding)
        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()



    def forward(self, x):
        identity = x
        identity = self.projector(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += identity
        x = self.relu(x)
        x = self.dropout(x)
        return x


In [26]:
class Net(nn.Module):
    def __init__(self , print_shape=True):
        self.print_shape = print_shape
        dropoutr = 0.1
        super(Net, self).__init__()
        self.layers1 = nn.ModuleList( [
        nn.Conv3d(2, 64, (17,1,1)),
        nn.BatchNorm3d(64),
        nn.ReLU(),
        nn.Dropout(dropoutr)])
        self.layersRES = nn.ModuleList( [
        ProjectedResBlock(64,128, (3,3), 1, 1, dropoutr),
        ResBlock(128,128, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(128,256, (3,3), 1, 1, dropoutr),

        ResBlockBN(256, (3,3), 1, 1, dropoutr),
        ResBlockBN(256, (3,3), 1, 1, dropoutr),
        ProjectedResBlockBN(256, (3,3), 1,0, dropoutr),
        ResBlockBN(512, (3,3), 1, 1, dropoutr),
        ResBlockBN(512, (3,3), 1, 1, dropoutr),
        ProjectedResBlockBN(512, (3,3), 1, 0, dropoutr),
        ResBlockBN(1024, (3,3), 1, 1, dropoutr),
        ResBlockBN(1024, (3,3), 1, 1, dropoutr),
        ProjectedResBlockBN(1024, (3,3), 1, 1, dropoutr),
        ResBlockBN(2048, (3,3), 1, 1, dropoutr),
        ResBlockBN(2048, (3,3), 1, 1, dropoutr),
        ProjectedResBlockBN(2048, (3,3), 1, 1, dropoutr),
        nn.AdaptiveAvgPool2d(1)])

        self.layers = nn.ModuleList([
        nn.Linear(4096, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(1024, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(1024, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(512, dir_number)])

        for l in self.layers1:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()


        for l in self.layers:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()
            elif isinstance(l, nn.Conv2d):
                n = l.kernel_size[0] * l.kernel_size[1] * l.out_channels
                l.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(l, nn.Linear):
                nn.init.xavier_normal(l.weight)

    def forward(self, x):
        x = x[:,0,:,:,:,:]
        for l in self.layers1:
            x = l(x)
            if  ( isinstance(l , ProjectedResBlock) or  isinstance(l , ResBlock) or  isinstance(l , ProjectedResBlockBN) or  isinstance(l , ResBlockBN) or isinstance(l , nn.Dropout) ) and self.print_shape:
                print(x.shape)
        x = x[:,:,0,:,:]
        for l in self.layersRES :
            x = l(x)
            if  (isinstance(l , ProjectedResBlock) or  isinstance(l , ResBlock) or isinstance(l , ProjectedResBlockBN) or  isinstance(l , ResBlockBN) or  isinstance(l , nn.Dropout) ) and self.print_shape:
                print(x.shape)
        x = torch.flatten(x, 1)
        for l in self.layers:
            x = l(x)
            if isinstance(l , nn.Dropout) and self.print_shape:
                print(x.shape)
        return x

In [27]:
net = Net().to(dev)
criterion = nn.CrossEntropyLoss().to(dev)


<ipython-input-26-f02febb1a319>:63: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(l.weight)


In [28]:
net = net.to(dev)
print(net(a))

torch.Size([32, 64, 1, 148, 73])
torch.Size([32, 128, 74, 37])
torch.Size([32, 128, 74, 37])
torch.Size([32, 256, 37, 19])
torch.Size([32, 256, 37, 19])
torch.Size([32, 256, 37, 19])
torch.Size([32, 512, 18, 9])
torch.Size([32, 512, 18, 9])
torch.Size([32, 512, 18, 9])
torch.Size([32, 1024, 8, 4])
torch.Size([32, 1024, 8, 4])
torch.Size([32, 1024, 8, 4])
torch.Size([32, 2048, 4, 2])
torch.Size([32, 2048, 4, 2])
torch.Size([32, 2048, 4, 2])
torch.Size([32, 4096, 2, 1])
torch.Size([32, 2048])
torch.Size([32, 1024])
torch.Size([32, 1024])
torch.Size([32, 512])
tensor([[-0.1882,  1.7151,  2.1043, -0.3966,  1.1039,  0.0611, -0.3857, -0.3083],
        [ 1.9770,  0.3984,  0.5859,  0.4077,  1.3415, -0.4722,  1.1419,  0.1630],
        [-0.6058,  2.9077, -0.6191,  2.2182,  1.1546,  0.7012, -0.1659, -0.4965],
        [-0.4701,  1.6985,  1.1861,  1.3479,  1.3419,  0.4070, -0.4783, -2.1809],
        [ 1.5447,  0.2909,  2.1034,  0.2517,  1.4737,  0.0995, -1.1926, -0.1200],
        [ 1.7794,  0.4169,

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
# exp_lr_scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/21DRESTNET.pt'

In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
    for i, data in enumerate(trainloader, 0):
      try:
        inputs, labels = data
        inputs = inputs.float().to(dev)
        labels = labels.to(dev)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
        if i % 10000 == 9999:    # print every 2000 mini-batches
               torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "/content/drive/MyDrive/21DRESNET.pt")
      except Exception as e:
        print(e)



print('Finished Training')

[1,   200] loss: 2.348
[1,   400] loss: 2.205
[1,   600] loss: 2.169
[1,   800] loss: 2.114
[1,  1000] loss: 2.091
[1,  1200] loss: 2.073
[1,  1400] loss: 2.064
[1,  1600] loss: 2.014
[1,  1800] loss: 2.000
[1,  2000] loss: 1.995
[1,  2200] loss: 1.964
[1,  2400] loss: 1.965
[1,  2600] loss: 1.967
[1,  2800] loss: 1.960
[1,  3000] loss: 1.940
[1,  3200] loss: 1.946
[1,  3400] loss: 1.912
[1,  3600] loss: 1.913
[1,  3800] loss: 1.902
[1,  4000] loss: 1.909
[1,  4200] loss: 1.889
[1,  4400] loss: 1.900
[1,  4600] loss: 1.873
[1,  4800] loss: 1.888
[1,  5000] loss: 1.882
[1,  5200] loss: 1.877
[1,  5400] loss: 1.873
[1,  5600] loss: 1.841
[1,  5800] loss: 1.852
[1,  6000] loss: 1.853
[1,  6200] loss: 1.844
[1,  6400] loss: 1.824
[1,  6600] loss: 1.824
[1,  6800] loss: 1.835
[1,  7000] loss: 1.797
[1,  7200] loss: 1.801
[1,  7400] loss: 1.763
[1,  7600] loss: 1.733
[1,  7800] loss: 1.727
[1,  8000] loss: 1.716
[1,  8200] loss: 1.690
[1,  8400] loss: 1.683
[1,  8600] loss: 1.692
[1,  8800] 

In [ ]:
net = torch.load("compressed2.pt")
net = net.to(dev)

In [ ]:
torch.save(net.state_dict, "compressed2.pt")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i in range(100):
        data = next(iter(trainloader))
        images, labels = data
        images = images.float().to(dev)
        labels = labels.to(dev)
        outputs = net(images)
        predicted = torch.argmax(torch.softmax(outputs.data, 1),axis =1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network : %d %%' % (
    100 * correct / total))

Accuracy of the network : 95 %
